## **Cleaning and ordering ***new_people_count_17_12_2024*****

In [5]:
import pandas as pd
import numpy as np

people_count = pd.read_csv("count_people_sorted_17_12_2024.csv")

# Convert the 'time' column to datetime for accurate sorting
people_count['time'] = pd.to_datetime(people_count['time'])

# Sort the DataFrame in descending order by 'time'
people_count_sorted = people_count.sort_values(by='time', ascending=False)

people_count_sorted.to_csv("new_people_count_12_12_2024.csv", index=False)

people_count_sorted.head()

,time,Image Name,Number of People
5091,2024-12-17 12:00:00,2024-12-17-12-00-image.png,0
5090,2024-12-17 11:50:00,2024-12-17-11-50-image.png,0
5089,2024-12-17 11:40:00,2024-12-17-11-40-image.png,0
5088,2024-12-17 11:30:00,2024-12-17-11-30-image.png,0
5087,2024-12-17 11:20:00,2024-12-17-11-20-image.png,0


In [6]:
# Count the rows where 'Number of People' equals 0
count_zero_people = (people_count['Number of People'] == 0).sum()

# Display the count
print(f"Number of rows where 'Number of People' is 0: {count_zero_people}")

Number of rows where 'Number of People' is 0: 4550


## **Cleaning and ordering ***all_sensor_data_17_12_2024.csv*****

In [16]:
# Read the CSV file
sensor_data = pd.read_csv("Plateau principal18_09_2024__17_12_2024.csv", delimiter=';')

# Rename columns
sensor_data.rename(columns={"Timezone : Europe/Paris": "time"}, inplace=True)
sensor_data.rename(columns={"CO2": "co2_value"}, inplace=True)
sensor_data.rename(columns={"Noise": "sound_value"}, inplace=True)

# Drop unwanted columns
sensor_data = sensor_data.drop(columns = ["Timestamp", "Temperature", "Humidity", "Pressure"])

# Convert 'time' column to datetime
sensor_data['time'] = pd.to_datetime(sensor_data['time'])

# **Round 'time' to the nearest lower 5-minute interval**
sensor_data['time'] = sensor_data['time'].dt.floor('5T')

# Filter out rows where both 'co2_value' and 'sound_value' are 0
sensor_data_filtered = sensor_data[(sensor_data['co2_value'] != 0) | (sensor_data['sound_value'] != 0)]

# Sort the DataFrame in descending order by 'time'
sensor_data_sorted = sensor_data_filtered.sort_values(by='time', ascending=False)

# Save to a new CSV
sensor_data_sorted.to_csv("cleaned_all_sensor_data_17_12_2024.csv", index=False)

# Display the sorted DataFrame
sensor_data_sorted.head()


C:\Users\dewmi\AppData\Local\Temp\ipykernel_27072\278557844.py:13: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  sensor_data['time'] = pd.to_datetime(sensor_data['time'])
C:\Users\dewmi\AppData\Local\Temp\ipykernel_27072\278557844.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  sensor_data['time'] = sensor_data['time'].dt.floor('5T')


,time,co2_value,sound_value
25837,2024-12-17 17:30:00,420.0,49.0
25836,2024-12-17 17:25:00,420.0,47.0
25835,2024-12-17 17:20:00,443.0,47.0
25834,2024-12-17 17:15:00,437.0,50.0
25833,2024-12-17 17:10:00,437.0,51.0


## Adding ventillation data to the sound, co2 dataset

### Extraction

In [17]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Sensor ID for Ventilation
sensor_id = 213  # Ventilation sensor

# Base URL for the API
base_url = "https://preprodapi.mde.epf.fr/measure.php"

# Number of rows to retrieve (or fetch all available data if API allows)
row_count = 120000  # Row Count

# List to hold ventilation data
ventilation_data = []

# Set parameters to fetch all the data for the ventilation sensor
params = {
    'id': sensor_id,
    'row_count': row_count  # Specify the number of records to fetch
}

# Make the request to the API
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Adjust extraction based on actual response format
    for entry in data:
        if "created" in entry and "value" in entry:
            ventilation_data.append({
                "time": entry["created"],  # Timestamp of the reading
                "ventilation_value": entry["value"]  # Sensor value
            })
        else:
            print(f"Skipping entry without 'created' or 'value': {entry}")
else:
    print(f"Failed to retrieve data for Sensor ID {sensor_id}")

# Load the ventilation data into a pandas DataFrame
df = pd.DataFrame(ventilation_data)

# Convert 'time' column to datetime
df['time'] = pd.to_datetime(df['time'])

# Date when the rounding rule changed
change_date = datetime(2023, 6, 19, 13, 14, 27)

# Function to round timestamps based on the change date
def round_time(row):
    if row['time'] < change_date:
        # Round to nearest 5 minutes
        delta = row['time'].minute % 5
        rounded_time = row['time'] - timedelta(minutes=delta)
    else:
        # Round to nearest 10 minutes
        delta = row['time'].minute % 10
        rounded_time = row['time'] - timedelta(minutes=delta)
        
    return rounded_time.replace(second=0, microsecond=0)

# Apply the rounding function to the 'time' column
df['time'] = df.apply(round_time, axis=1)

# Get today's date in DD_MM_YYYY format
today_date = datetime.now().strftime("%d_%m_%Y")

# Create the filename dynamically
filename = f"ventilation_data_17_12_2024.csv"

# Write the data to a CSV file
df[['time', 'ventilation_value']].to_csv(filename, index=False)

print(f"CSV file '{filename}' created successfully.")


Skipping entry without 'created' or 'value': {'general': {'id_system_sensor': '213', 'name': 'EL_DF_CO_VA_Cor_RunMode', 'description': 'Modbus:|0=Stopped|1=Starting up|2=Starting reduced speed|3=Starting full speed|4=Starting normal run|5=Normal run|6=Support control heating|7=Support control cooling|8=CO_{2} run|9=Night cooling|10=Full speed stop|11=Stopping fan|BACnet:|1=Stopped|2=Starting up|3=Starting reduced speed|4=Starting full speed|5=Starting normal run|6=Normal run|7=Support control heating|8=Support control cooling|9=CO_{2} run|10=Night cooling|11=Full speed stop|12=Stopping fan', 'unite': '213', 'project': {'id_project': '69', 'name': 'EL_DF_CO_VA', 'description': 'VentActual', 'project': {'id_project': '68', 'name': 'EL_DF_CO', 'description': 'REGULATEUR CORRIGO', 'project': {'id_project': '67', 'name': 'EL_DF', 'description': 'DOUBLE FLUX', 'project': {'id_project': '1', 'name': 'EL', 'description': 'ENERGYLAB'}}}}}}
CSV file 'ventilation_data_17_12_2024.csv' created succ

## Merging

In [3]:
import pandas as pd

df_vent = pd.read_csv('ventilation_data_17_12_2024.csv', parse_dates=['time'])

df_sensors = pd.read_csv(f'Sound_Co2_17_12_2024.csv', parse_dates=['time'])


# Merge the DataFrames on the 'time' column using an inner join
merged_df = pd.merge(df_vent[['time','ventilation_value']], df_sensors[['time','sound_value', 'co2_value']], on='time', how='inner')

merged_df.to_csv(f'CO2_vent_sound_17_12_2024.csv', index=False)

merged_df.head()

,time,ventilation_value,sound_value,co2_value
0,2024-12-17 17:30:00,5,49.0,420.0
1,2024-12-17 17:20:00,5,47.0,443.0
2,2024-12-17 17:10:00,5,51.0,437.0
3,2024-12-17 17:00:00,5,50.0,434.0
4,2024-12-17 16:50:00,5,47.0,464.0
